In [7]:
import pandas as pd
temp = pd.read_excel("/content/최종일까4.xlsx")
im = pd.read_excel("/content/drive/My Drive/Data/movie/1970_2019_imdb2.xlsx")
df = pd.read_excel("/content/drive/My Drive/Data/1970_2019_영화전체.xlsx")


FileNotFoundError: ignored

# Word to Score

In [ ]:
!pip install konlpy
!pip install googletrans
!pip install gensim

In [ ]:
# Read Review Data
from tqdm import tqdm
import nltk
import re 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm import tqdm
import re 
import tqdm.notebook as tq

from sklearn import svm
from keras.utils import get_file
import os
import gensim
import subprocess
import numpy as np
import random
import requests
import pandas as pd
from IPython.core.pylabtools import figsize
import csv
import matplotlib.pyplot as plt
import gensim.downloader as api
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from numpy.linalg import norm
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import konlpy
from konlpy.tag import Okt
from konlpy.tag import Kkma
import re

import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

tqdm.pandas(position=0, leave=True)
from googletrans import Translator
translator = Translator()

model = api.load('glove-wiki-gigaword-200')

nltk.download('wordnet')
nltk.download('vader_lexicon')

hello 여보세요
there 그곳에


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
# Data
# English Preprocessing ----------------------------------------------------------------
print("Start English Preprocessing...")
def trim_en(text):
    text = str(text)
    #text =text.lower()
    text=re.sub("[(<.*?>)'']"," ",text)
    text=re.sub("(\\W|\\d)"," ",text)
    text=text.strip()
    
    return text

lemm =  WordNetLemmatizer()
sa = SentimentIntensityAnalyzer()

im.리뷰 = im.리뷰.progress_apply(trim_en)
im.리뷰 = im.리뷰.progress_apply(lambda s:" ".join([lemm.lemmatize(w) for w in str(s).split()]) )


# Sentimental Analysis ----------------------------------------------------------------
print("Start  Sentimental Analysis...")
im['neg'] = 0
im['pos'] = 0
im['neu'] = 0
im['compound'] = 0

for i in tq.tqdm(range(len(im))):
    scores = sa.polarity_scores(str(im.loc[i, '리뷰']))
    im.loc[i,'neg'] = scores['neg']
    im.loc[i,'pos'] = scores['pos']
    im.loc[i,'neu'] = scores['neu']
    im.loc[i,'compound'] = scores['compound']

# Comment Analysis ----------------------------------------------------------------
print("Start  Comment Analysis...")
words = ["스토리", "음악", "연출", "배우","연기"]
translator = Translator()
trans = translator.translate(words, dest="en")

lst2 = [model.most_similar(positive=[lemm.lemmatize(word.text.lower())],topn=50) for word in trans ]
lst2 = [ [trans[k].text] + [j[0] for j in i]  for k,i in enumerate(lst2)]

values = []
values_pos = []
values_neg = []

for i in tq.tqdm(range(len(im))):
    review1 = im.loc[i, '리뷰']
    value = [0.001 for i in range(len(lst2))]
    value_pos = [0.001 for i in range(len(lst2))]
    value_neg = [0.001 for i in range(len(lst2))]
    string_split = str(review1).split()
    for k2, i in enumerate(string_split):
        for k,j in enumerate(lst2):
            if i in j:
                value[k] +=1
                left = max([0, k2-10])
                right = k2+10
                window = " ".join(string_split[left:right])
                scores = sa.polarity_scores(window)
                if scores['compound']>0:
                    value_pos[k] += scores['pos']
                if scores['compound']<0:
                    value_neg[k] += scores['neg']

    #Normalize  
    v = [value_pos[i]/value[i] for i in range(len(value_pos))]
    values_pos.append(v)
    # else
    values_pos.append(value_pos)


    #Normalize
    #v = [value_neg[i]/value[i] for i in range(len(value_neg))]
    #values_neg.append(v)
    values_neg.append(value_neg)

    #Normalize
    #SUM = sum(value)
    #v = [i/SUM for i in value]
    #values.append(v)
    values.append(value)

labels = words
temp1 = pd.DataFrame(values, columns = [i+"_com" for i in  labels])
temp2 = pd.DataFrame(values_pos, columns = [i+"_pos" for i in  labels])
temp3 = pd.DataFrame(values_neg, columns = [i+"_neg" for i in  labels])

data1 = pd.concat([im[['neg', 'pos','neu' ,'compound']], temp1, temp2, temp3],axis=1)
data1.info()
data1.head()

Start English Preprocessing...



Start  Sentimental Analysis...



Start  Comment Analysis...


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5825 entries, 0 to 5824
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   neg       5825 non-null   float64
 1   pos       5825 non-null   float64
 2   neu       5825 non-null   float64
 3   compound  5825 non-null   float64
 4   스토리_com   5825 non-null   float64
 5   음악_com    5825 non-null   float64
 6   연출_com    5825 non-null   float64
 7   배우_com    5825 non-null   float64
 8   연기_com    5825 non-null   float64
 9   스토리_pos   5825 non-null   float64
 10  음악_pos    5825 non-null   float64
 11  연출_pos    5825 non-null   float64
 12  배우_pos    5825 non-null   float64
 13  연기_pos    5825 non-null   float64
 14  스토리_neg   5825 non-null   float64
 15  음악_neg    5825 non-null   float64
 16  연출_neg    5825 non-null   float64
 17  배우_neg    5825 non-null   float64
 18  연기_neg    5825 non-null   float64
dtypes: float64(19)
memory usage: 864.8 KB


,neg,pos,neu,compound,스토리_com,음악_com,연출_com,배우_com,연기_com,스토리_pos,음악_pos,연출_pos,배우_pos,연기_pos,스토리_neg,음악_neg,연출_neg,배우_neg,연기_neg
0,0.0,0.000,1.000,0.0000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001
1,0.0,0.000,1.000,0.0000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001
2,0.1,0.196,0.704,0.9999,281.001,29.001,19.001,158.001,62.001,39.562,4.993,3.610,23.860,6.999,11.516,1.132,0.301,6.012,3.492
3,0.0,0.000,1.000,0.0000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001
4,0.0,0.000,0.000,0.0000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001


In [ ]:
def cosim(x,y):
    denom = norm(x)* norm(y)
    if denom==0:
        return 0
    return np.dot(x,y.T)/(norm(x)* norm(y))

def nuclear_norm(x,y):
    return np.sum(np.abs(x-y))

def spectral_norm(x,y):
    return np.max(np.abs(x-y))

def l2_norm(x,y):
    return np.sqrt(np.sum((x-y)**2))

def rbf_kernel(x,y):
    return np.exp(-1/2*(np.sum((x-y)**2)))

def most_similar(X, x, n, method="cosine"):
    if method=="cosine":
        lst = [(cosim(X[x, :], X[i,:]), i) for i in range(len(X))]
        lst.sort(reverse=True)
    if method=="nuclear":
        lst = [(nuclear_norm(X[x, :], X[i,:]), i) for i in range(len(X))]
        lst.sort(reverse=False)
    if method=="spectral":
        lst = [(spectral_norm(X[x, :], X[i,:]), i) for i in range(len(X))]
        lst.sort(reverse=False)
    if method=="l2":
        lst = [(l2_norm(X[x, :], X[i,:]), i) for i in range(len(X))]
        lst.sort(reverse=False)    
    if method=="rbf":
        lst = [(rbf_kernel(X[x, :], X[i,:]), i) for i in range(len(X))]
        lst.sort(reverse=False)    
    return lst[:n]


# Korean 


okt = Okt()
kkma = Kkma()
def trim_ko(x):
    text=re.sub("[(<.*?>)/'…“”']"," ",x)
    #text=re.sub("(\\W|\\d)"," ",text)
    
    #text = kkma.sentences(text)
    #print(text)
    text = " ".join([w[0] for w in okt.pos(text) if w[1]=="Noun" or w[1]=='Verb' or w[1]=="Adjective"])
    #textx= " ".join([w[0] for w in kkma.pos(text) if w[1]=='NNG' or w[1]=='VA' or w[1]=="MAG"  or w[1]=='MAC'])

    #remove whitespace
    text=text.strip()
    return text


df.스토리_ko.fillna("", inplace=True)
df.스토리_ko = df.스토리_ko.astype(str)
df.genres = df.genres.astype(str)

df.스토리_ko = df.스토리_ko.apply(lambda x:x.replace("\n"," "))
df.스토리_ko = df.스토리_ko.progress_apply(trim_ko)

corpus = df.스토리_ko[(df.스토리_ko.notnull()) & (df.스토리_ko.apply(lambda x: True if len(x)>3 else False))]
corpus = corpus.reset_index()

vectorizer = TfidfVectorizer(max_features=2000)
X = vectorizer.fit_transform(corpus.스토리_ko.values)
X = X.toarray()

pca = PCA(n_components=5)
pca.fit(X)

new_X = pca.fit_transform(X)

kmeans = KMeans(n_clusters=10, random_state=0).fit(new_X)

data2 = pd.DataFrame([corpus['index'], new_X[:,0], new_X[:,1], new_X[:,2],new_X[:,3], new_X[:,4], kmeans.labels_]).T
data2.columns=['df_index','x0','x1','x2','x3','x4','label']
data2.df_index = data2.df_index.astype(int)
data2.label = data2.label.astype(int)

print("Kmeans...")
for i in tqdm(range(kmeans.cluster_centers_.shape[0])):
    data2['center_angle'+str(i+1)] = 0
    for j in range(len(data2)):
        data2.loc[j, 'center_angle'+str(i+1)] =  cosim(kmeans.cluster_centers_[i,:], data2.iloc[j, 1:kmeans.cluster_centers_.shape[1]+1].values )

data2 = data2.set_index('df_index')

In [ ]:
temp = temp[['index', 'movienm', 'showtm', 'prdtyear', 'domestic', 'mpaa', 'raters',
       'ratings', 'kor_revenue', 'kor_audience', 'sf', 'family', 'performance',
       'horror', 'etc', 'documentary', 'drama', 'romance', 'musical',
       'mystery', 'crime', 'history', 'western', 'adult', 'thriller',
       'animation', 'action', 'adventure', 'war', 'comedy', 'fantasy',
       'director_appearance', 'director_revenue', 'distributor_share',
       'opendt_quarter', 'year_gap', 'showtypes_num', 'actor_score']]

temp = temp.set_index('index')
temp = temp.join(data1)
temp = temp.join(data2)
temp

,movienm,showtm,prdtyear,domestic,mpaa,raters,ratings,kor_revenue,kor_audience,sf,family,performance,horror,etc,documentary,drama,romance,musical,mystery,crime,history,western,adult,thriller,animation,action,adventure,war,comedy,fantasy,director_appearance,director_revenue,distributor_share,opendt_quarter,year_gap,showtypes_num,actor_score,neg,pos,neu,compound,스토리_com,음악_com,연출_com,배우_com,연기_com,스토리_pos,음악_pos,연출_pos,배우_pos,연기_pos,스토리_neg,음악_neg,연출_neg,배우_neg,연기_neg,x0,x1,x2,x3,x4,label,center_angle1,center_angle2,center_angle3,center_angle4,center_angle5,center_angle6,center_angle7,center_angle8,center_angle9,center_angle10
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,드라이브,100,2011,35061555,15,228016,7.2,156237980,D,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1.168718e+07,0.0016,4,0,2,52.00,0.100,0.196,0.704,0.9999,281.001,29.001,19.001,158.001,62.001,39.562,4.993,3.610,23.860,6.999,11.516,1.132,0.301,6.012,3.492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,조조 래빗,108,2019,33370906,12,250206,7.9,965083200,C,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,2,1.161431e+08,0.0000,1,1,1,92.25,0.000,0.000,0.000,0.0000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,-0.007540,-0.075842,0.022933,-0.044247,-0.063058,3.0,0.038814,-0.716655,0.078862,0.339878,-0.375964,0.672698,-0.067346,0.647321,-0.458175,0.024889
10,존 윅 - 리로드,122,2017,92029184,18,351231,7.5,2231583601,C,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,2,3.067639e+07,0.0407,1,0,2,6.00,0.142,0.183,0.675,0.9998,505.001,12.001,11.001,209.001,67.001,60.987,1.521,2.413,24.304,5.455,39.561,1.431,0.211,18.843,8.441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,메멘토,113,2000,25544867,15,1096788,8.4,325278300,D,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,4,8.514956e+06,0.0020,3,1,2,32.00,0.111,0.152,0.737,0.9996,315.001,0.001,15.001,113.001,79.001,36.683,0.001,2.659,14.499,8.829,15.601,0.001,0.087,4.838,5.128,0.046876,0.049038,0.000488,-0.058251,0.002059,3.0,0.135686,0.410639,-0.063341,-0.127295,0.771331,-0.067157,-0.813821,-0.030215,0.236628,-0.900306
25,존 윅: 특별판,107,2014,43037835,18,519268,7.4,81322120,D,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1.434594e+07,0.0407,1,3,1,50.00,0.000,0.000,0.000,0.0000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,-0.010228,-0.049613,-0.044464,-0.046167,0.042366,7.0,-0.046307,0.082564,-0.085325,-0.468822,-0.140854,-0.452838,-0.074363,0.905896,-0.665752,0.224040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5780,쿼터매인,99,1986,3751699,12,8643,4.6,147000,F,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1.250566e+06,0.0000,1,1,1,0.00,0.000,0.000,1.000,0.0000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5783,크로커다일 던디2,112,1988,109306210,15,844,3.3,39000,F,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,3.643540e+07,0.1035,2,2,1,47.50,0.000,0.000,1.000,0.0000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.043570,0.108399,0.023611,-0.001122,-0.006706,8.0,0.117158,0.292950,-0.140839,0.044887,0.686769,0.060973,-0.373589,-0.739734,0.741932,-0.646291
5788,탱고와 캐쉬,100,1989,63408614,15,307534,7.5,266000,F,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0.000000e+00,0.1519,3,1,1,0.00,0.000,0.000,1.000,0.0000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.045302,0.193306,-0.049250,-0.001276,0.041615,4.0,0.023809,0.518710,-0.346564,-0.421523,0.792947,-0.476607,-0.240445,-0.544054,0.684770,-0.420068


In [ ]:
temp[temp['movienm']=='겨울왕국']

,movienm,showtm,prdtyear,domestic,mpaa,raters,ratings,kor_revenue,kor_audience,sf,family,performance,horror,etc,documentary,drama,romance,musical,mystery,crime,history,western,adult,thriller,animation,action,adventure,war,comedy,fantasy,director_appearance,director_revenue,distributor_share,opendt_quarter,year_gap,showtypes_num,actor_score,neg,pos,neu,compound,스토리_com,음악_com,연출_com,배우_com,연기_com,스토리_pos,음악_pos,연출_pos,배우_pos,연기_pos,스토리_neg,음악_neg,연출_neg,배우_neg,연기_neg,x0,x1,x2,x3,x4,label,center_angle1,center_angle2,center_angle3,center_angle4,center_angle5,center_angle6,center_angle7,center_angle8,center_angle9,center_angle10
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1777,겨울왕국,108,2013,400738009,12,560064,7.4,82521307080,A,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,1,2,1.335793e+08,0.1694,1,1,4,0.0,0.046,0.268,0.686,1.0,167.001,26.001,7.001,64.001,7.001,37.281,4.31,1.25,13.223,0.338,2.219,0.816,0.001,0.977,0.297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
temp[temp['movienm']=='어스']

,movienm,showtm,prdtyear,domestic,mpaa,raters,ratings,kor_revenue,kor_audience,sf,family,performance,horror,etc,documentary,drama,romance,musical,mystery,crime,history,western,adult,thriller,animation,action,adventure,war,comedy,fantasy,director_appearance,director_revenue,distributor_share,opendt_quarter,year_gap,showtypes_num,actor_score,neg,pos,neu,compound,스토리_com,음악_com,연출_com,배우_com,연기_com,스토리_pos,음악_pos,연출_pos,배우_pos,연기_pos,스토리_neg,음악_neg,연출_neg,배우_neg,연기_neg,x0,x1,x2,x3,x4,label,center_angle1,center_angle2,center_angle3,center_angle4,center_angle5,center_angle6,center_angle7,center_angle8,center_angle9,center_angle10
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
412,어스,116,2019,175084580,15,209068,6.9,12586771104,B,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,1.170417e+08,0.1173,1,0,2,31.92,0.135,0.188,0.677,0.9974,123.001,8.001,5.001,50.001,17.001,14.935,1.298,0.367,6.502,3.016,10.415,0.396,0.997,5.195,0.6,0.002338,-0.207558,-0.032139,-0.234639,-0.195512,7.0,0.078493,-0.624001,-0.158586,0.040723,-0.08404,0.463621,-0.234789,0.786541,-0.390929,-0.127646


In [ ]:

values = """SF	6.897317	0.056442	0.032214	0.575971	0.335372	0.037166	0.079923	0.469141	0.413771
가족	8.631818	0.047048	0.013229	0.582215	0.357508	0.033949	0.069785	0.477073	0.419193
공연	8.010000	0.288701	0.147626	0.268308	0.295364	0.191173	0.064015	0.364478	0.380334
공포(호러)	5.551064	0.102583	0.056846	0.511650	0.328922	0.044389	0.076611	0.494285	0.384716
다큐멘터리	8.198421	0.151912	0.016448	0.422084	0.409556	0.082581	0.104008	0.336566	0.476845
드라마	8.111493	0.070156	0.027030	0.518252	0.384562	0.050804	0.075190	0.427861	0.446145
멜로/로맨스	7.978472	0.074606	0.014942	0.567023	0.343429	0.047151	0.070064	0.447307	0.435478
뮤지컬	8.154000	0.348072	0.045269	0.362816	0.243842	0.221074	0.094741	0.333895	0.350290
미스터리	6.628621	0.038032	0.038757	0.536515	0.386696	0.036544	0.080871	0.483196	0.399389
범죄	7.117439	0.046644	0.037167	0.567117	0.349072	0.033048	0.075974	0.449683	0.441294
사극	7.976667	0.021509	0.078599	0.533271	0.366621	0.037533	0.047632	0.422369	0.492466
서부극(웨스턴)	6.722500	0.072643	0.067593	0.536934	0.322830	0.026449	0.063326	0.443889	0.466336
스릴러	6.707849	0.062616	0.061160	0.513240	0.362984	0.030545	0.074922	0.472481	0.422051
애니메이션	8.391441	0.079559	0.019899	0.505829	0.394713	0.063208	0.090650	0.463264	0.382878
액션	6.810346	0.055686	0.045305	0.572117	0.326892	0.036780	0.076935	0.474533	0.411752
어드벤처	7.663667	0.052433	0.034778	0.540306	0.372482	0.043553	0.079389	0.469152	0.407905
전쟁	7.756000	0.036516	0.016633	0.615999	0.330852	0.045915	0.082050	0.434713	0.437322
코미디	7.222879	0.067258	0.023799	0.597824	0.311120	0.040729	0.073608	0.454548	0.431116
판타지	7.048485	0.040566	0.024315	0.583863	0.351256	0.036027	0.075312	0.481063	0.407599"""
values = values.split('\n')
values = [i.split('\t') for i  in values]
genres = [i[0] for i in values]
values = [i[1:] for i in values]
df = pd.DataFrame(values, columns="""평점	v_음악_kr	v_연출_kr	v_배우_kr	v_연기_kr	v_음악_en	v_연출_en	v_배우_en	v_연기_en""".split('\t'), index= genres)
df = df.astype(float)
for i in range(4):
    df['diff' +str(i)] = df.iloc[:,i+1] - df.iloc[:,i+5] 
df

,평점,v_음악_kr,v_연출_kr,v_배우_kr,v_연기_kr,v_음악_en,v_연출_en,v_배우_en,v_연기_en,diff0,diff1,diff2,diff3
SF,6.897317,0.056442,0.032214,0.575971,0.335372,0.037166,0.079923,0.469141,0.413771,0.019276,-0.047709,0.106830,-0.078399
가족,8.631818,0.047048,0.013229,0.582215,0.357508,0.033949,0.069785,0.477073,0.419193,0.013099,-0.056556,0.105142,-0.061685
공연,8.010000,0.288701,0.147626,0.268308,0.295364,0.191173,0.064015,0.364478,0.380334,0.097528,0.083611,-0.096170,-0.084970
공포(호러),5.551064,0.102583,0.056846,0.511650,0.328922,0.044389,0.076611,0.494285,0.384716,0.058194,-0.019765,0.017365,-0.055794
다큐멘터리,8.198421,0.151912,0.016448,0.422084,0.409556,0.082581,0.104008,0.336566,0.476845,0.069331,-0.087560,0.085518,-0.067289
드라마,8.111493,0.070156,0.027030,0.518252,0.384562,0.050804,0.075190,0.427861,0.446145,0.019352,-0.048160,0.090391,-0.061583
멜로/로맨스,7.978472,0.074606,0.014942,0.567023,0.343429,0.047151,0.070064,0.447307,0.435478,0.027455,-0.055122,0.119716,-0.092049
뮤지컬,8.154000,0.348072,0.045269,0.362816,0.243842,0.221074,0.094741,0.333895,0.350290,0.126998,-0.049472,0.028921,-0.106448
미스터리,6.628621,0.038032,0.038757,0.536515,0.386696,0.036544,0.080871,0.483196,0.399389,0.001488,-0.042114,0.053319,-0.012693
범죄,7.117439,0.046644,0.037167,0.567117,0.349072,0.033048,0.075974,0.449683,0.441294,0.013596,-0.038807,0.117434,-0.092222


In [ ]:
df.to_excel("장르별감정차이.xlsx", index=False)